In [1]:
import stim
import numpy as np
from numpy.linalg import matrix_power, matrix_rank
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
import os
from tqdm import tqdm
import galois
import qldpc

In [2]:
HX = np.array([[0, 0, 0, 1, 1, 1, 1], 
               [0, 1, 1, 0, 0, 1, 1], 
               [1, 0, 1, 0, 1, 0, 1]
               ])
HZ = np.array([[0, 0, 0, 1, 1, 1, 1], 
               [0, 1, 1, 0, 0, 1, 1], 
               [1, 0, 1, 0, 1, 0, 1]
               ])
logical_x = np.array([0, 0, 0, 0, 1, 1, 1])

In [18]:
circuit = stim.Circuit()
circuit.append("R", [i for i in range(7)])
print(circuit)
# circuit.diagram("timeline-svg")
for stabilizer in HX:
    detector = []
    for i, qubit in enumerate(stabilizer):
        if qubit == 1:
            detector.append(stim.target_x(i))
            detector.append(stim.target_combiner())
    if detector != None:
        detector.pop()
    circuit.append_operation("MPP", detector)
for stabilizer in HZ:
    detector = []
    for i, qubit in enumerate(stabilizer):
        if qubit == 1:
            detector.append(stim.target_z(i))
            detector.append(stim.target_combiner())
    if detector != None:
        detector.pop()
    circuit.append_operation("MPP", detector)
circuit.append("DEPOLARIZE1", [i for i in range(7)], 0)
circuit.append("X_ERROR", [0, 1, 2], 0.1)


for stabilizer in HX:
    detector = []
    for i, qubit in enumerate(stabilizer):
        if qubit == 1:
            detector.append(stim.target_x(i))
            detector.append(stim.target_combiner())
    if detector != None:
        detector.pop()
    circuit.append_operation("MPP", detector)
for stabilizer in HZ:
    detector = []
    for i, qubit in enumerate(stabilizer):
        if qubit == 1:
            detector.append(stim.target_z(i))
            detector.append(stim.target_combiner())
    if detector != None:
        detector.pop()
    circuit.append_operation("MPP", detector)

for i in range(1, 7):
    circuit.append("DETECTOR", [stim.target_rec(-i), stim.target_rec(-i-6)])

target1 = [
    stim.target_x(0), 
    stim.target_combiner(),
    stim.target_x(1), 
    stim.target_combiner(),
    stim.target_x(2), 
    stim.target_combiner(),
    stim.target_x(3), 
    stim.target_combiner(),
    stim.target_x(4), 
    stim.target_combiner(),
    stim.target_x(5), 
    stim.target_combiner(),
    stim.target_x(6)
]
target2 = [
    stim.target_z(0), 
    stim.target_combiner(),
    stim.target_z(1), 
    stim.target_combiner(),
    stim.target_z(2), 
    stim.target_combiner(),
    stim.target_z(3), 
    stim.target_combiner(),
    stim.target_z(4), 
    stim.target_combiner(),
    stim.target_z(5), 
    stim.target_combiner(),
    stim.target_z(6)
]
circuit.append("M",[qbt for qbt in range(7)])
# circuit.append_operation("MPP", target1)
circuit.append("OBSERVABLE_INCLUDE", [stim.target_rec(-i-1) for i in range(7)], 1)
# circuit.append_operation("MPP", target2)
# circuit.append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], 2)

from bp_osd import BPOSD

# sampler = circuit.compile_detector_sampler()# sampler返回的是一个二维数组
# print(sampler.sample(shots=5))#第一次采样结果默认为False，因为奇偶性变化无法看出来
sampler = circuit.compile_detector_sampler()
dem = circuit.detector_error_model()
# print(dem)
decoder = BPOSD(dem, max_bp_iters = 20) #使用BP_OSD

# # sample 10 shots
N = 50
syndrome, actual_observables = sampler.sample(shots=N, separate_observables=True)
# print("Syndrome shape:", syndrome.shape)
# print("First few syndromes:", syndrome[:5])

predicted_observables = decoder.decode_batch(syndrome)
num_errors = np.sum(np.any(predicted_observables != actual_observables, axis=1))
# logical_error_rate.append(num_errors / N)
print(f'Error rate: {num_errors / N * 100}%')
# circuit.diagram("timeline-svg")

# circuit.append_operation("MPP", [
#     stim.target_x(0),
#     stim.target_combiner(),
#     stim.target_z(1),
#     stim.target_combiner(),
#     stim.target_z(2),
#     stim.target_combiner(),
#     stim.target_x(3),
# ])

R 0 1 2 3 4 5 6
Check matrix shape: (6, 3)
Error rate: 4.0%
